In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import random
from task_1c_v2 import cvr_op_scheme, cvr_tp_scheme

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] = True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4


In [2]:
# Load data
with open('Data/ALL_scenarios.json') as f:
    all_scenarios = json.load(f)

# Constants
OMEGA = 250  # Number of scenarios to sample
PI = 1 / OMEGA  # Probability of each sampled scenario - assumed to be equal
S = len(all_scenarios.keys()) - 1  # Total number of scenarios
T = 24  # Number of hours
WIND_CAPACITY = 200  # MWh

random.seed(123)

# Sample scenarios without replacement
in_sample_scenarios = random.sample(range(S), 250)

scenarios = {}

# Extract sampled scenarios from dictionary containing all scenarios
for i in range(len(in_sample_scenarios)):
    scenarios[str(i)] = all_scenarios[str(in_sample_scenarios[i])]
    scenarios[str(i)]['Original Index'] = in_sample_scenarios[i]

# Indices of all scenarios
all_indices = list(range(S))



In [3]:
alpha = 0.9
beta_values = np.arange(0,1 + 0.1, 0.1)

In [5]:

# Initialize lists to store results
out_sample_profits = []
expected_values = []

num_scenarios = [170, 250, 360, 430, 540]

for OMEGA in num_scenarios:
    in_sample_scenarios = random.sample(range(len(all_scenarios.keys()) - 1), OMEGA)
    
    scenarios = {}
    for i, idx in enumerate(in_sample_scenarios):
        scenarios[str(i)] = all_scenarios[str(idx)]
        scenarios[str(i)]['Original Index'] = idx
    
    # Indices of out-of-sample scenarios
    out_of_sample_indices = list(set(all_indices) - set(in_sample_scenarios))

    # Sample 950 out-of-sample scenarios without replacement
    out_sample_indices = random.sample(out_of_sample_indices, 950)
    print(out_sample_indices)

    # Extract out-of-sample scenarios from the dictionary containing all scenarios
    out_sample_scenarios = {}
    for i, idx in enumerate(out_sample_indices):
        out_sample_scenarios[str(i)] = all_scenarios[str(idx)]
        out_sample_scenarios[str(i)]['Original Index'] = idx

    # Run optimization for one-price
    results_per_beta_op, p_DA_values_per_beta_op = cvr_op_scheme(scenarios, WIND_CAPACITY, T, OMEGA, alpha, beta_values, minimize_printouts=False, mip_gap = 1e-4) 
    beta_val = 0.5
    optimal_DA_oneprice = np.array(p_DA_values_per_beta_op[beta_val])
    expected_profit = results_per_beta_op[beta_val]['Expected Profit']

    out_sample_earnings = 0

    for scenario_idx in out_sample_scenarios:
        scenario = out_sample_scenarios[scenario_idx]

        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_oneprice

        realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_oneprice

        price_coefficient = 1.2 * np.array(scenario['System Balance State']) + 0.9 * (1 - np.array(scenario['System Balance State']))
        y = realized_imbalance * price_coefficient * scenario['Spot Price [EUR/MWh]']

        payment_earnings = x + y
        
        out_sample_earnings += np.sum(payment_earnings)

        
    # Calculate average payment or earnings over out-of-sample scenarios
    average_out_sample_profits = out_sample_earnings / len(out_sample_scenarios)
    out_sample_profits.append(average_out_sample_profits)

    expected_values.append(expected_profit)
        
    print(f"Number of scenarios {OMEGA}  in-sample profits: {expected_profit}, out-sample profits: {average_out_sample_profits}")


[302, 1308, 1148, 853, 845, 1679, 422, 11, 142, 1656, 1426, 1336, 1586, 1367, 508, 1501, 353, 551, 120, 1931, 1116, 765, 753, 455, 1509, 872, 884, 1252, 144, 1199, 695, 1460, 1737, 848, 1557, 400, 1880, 1292, 943, 383, 729, 504, 798, 1193, 1625, 1312, 1311, 1530, 80, 411, 1702, 973, 1440, 1977, 1742, 1023, 1753, 1606, 240, 37, 1115, 1934, 1408, 1561, 126, 895, 140, 361, 1936, 209, 292, 748, 1456, 1993, 570, 541, 421, 1685, 550, 984, 49, 78, 156, 1134, 486, 1067, 733, 1232, 305, 1555, 363, 1649, 1933, 1663, 1071, 1304, 1505, 1402, 532, 251, 941, 239, 1029, 1991, 1483, 1664, 784, 1485, 162, 316, 1004, 1313, 1189, 517, 1839, 1239, 309, 556, 498, 548, 362, 188, 594, 1719, 1407, 685, 208, 1917, 526, 8, 1948, 574, 832, 812, 252, 534, 1997, 697, 28, 1802, 141, 1578, 1990, 1591, 625, 175, 1731, 1506, 1554, 1162, 1466, 1779, 1153, 882, 1871, 822, 744, 1040, 1395, 466, 1875, 972, 1086, 86, 165, 1754, 1722, 122, 237, 222, 1059, 870, 440, 1168, 1571, 1181, 1611, 1648, 21, 1226, 1626, 1491, 1975, 7

In [6]:
print(f"out-sample {out_sample_profits}")
print(f"In-sample {expected_values}")


out-sample [188895.36639470013, 179723.88977220148, 176348.42047378668, 181354.6437947203, 183587.03156882286]
In-sample [177081.64674808542, 177503.89134170668, 179141.89562433067, 180643.2309203232, 182922.080470185]


In [4]:
out_sample_profits_tp = []
expected_values_tp = []

num_scenarios = [170, 250, 360, 430, 540]

# Sample in-sample scenarios without replacement
for OMEGA in num_scenarios:
    in_sample_scenarios = random.sample(range(len(all_scenarios.keys()) - 1), OMEGA)
    
    scenarios = {}
    for i, idx in enumerate(in_sample_scenarios):
        scenarios[str(i)] = all_scenarios[str(idx)]
        scenarios[str(i)]['Original Index'] = idx
    
    # Indices of out-of-sample scenarios
    out_of_sample_indices = list(set(all_indices) - set(in_sample_scenarios))

    # Sample 950 out-of-sample scenarios without replacement
    out_sample_indices = random.sample(out_of_sample_indices, 950)

    # Extract out-of-sample scenarios from the dictionary containing all scenarios
    out_sample_scenarios = {}
    for i, idx in enumerate(out_sample_indices):
        out_sample_scenarios[str(i)] = all_scenarios[str(idx)]
        out_sample_scenarios[str(i)]['Original Index'] = idx

    # Run optimization for one-price
    results_per_beta_tp, p_DA_values_per_beta_tp = cvr_tp_scheme(scenarios, WIND_CAPACITY, T, OMEGA, alpha, beta_values, minimize_printouts=False, mip_gap = 1e-4) 
    beta_val = 0.5
    optimal_DA_twoprice = np.array(p_DA_values_per_beta_tp[beta_val])
    expected_profit = results_per_beta_tp[beta_val]['Expected Profit']

    out_sample_earnings_tp = 0

    for scenario_idx in out_sample_scenarios:
        scenario = out_sample_scenarios[scenario_idx]

        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_twoprice

        realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_twoprice

        price_coefficients = np.zeros(T)

        for t in range(T):
            if (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 0):
                price_coefficients[t] = 0.9
            elif (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 1):
                price_coefficients[t] = 1.0
            elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 0):
                price_coefficients[t] = -1.0
            elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 1):
                price_coefficients[t] = -1.2

        y = np.abs(realized_imbalance) * price_coefficients * scenario['Spot Price [EUR/MWh]']

        payment_earnings_tp = x + y

        out_sample_earnings_tp += np.sum(payment_earnings_tp)

    # Calculate average payment or earnings over out-of-sample scenarios
    average_out_sample_profits_tp = out_sample_earnings_tp / len(out_sample_scenarios)
    out_sample_profits_tp.append(average_out_sample_profits_tp)

    expected_values_tp.append(expected_profit)
        
    print(f"Number of scenarios {OMEGA}  in-sample profits function: {expected_profit}, out-sample profits: {average_out_sample_profits_tp}")

Solving for beta = 0.00...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-26


-----------------------------------------------
Objective value: 164923.82 EUR
Expected Profit: 164923.82 EUR
CVaR: 0.0000000000 EUR
VaR: 0.00 EUR
Eta values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 